In [1]:
# Import findspark and initialize.
import findspark
findspark.init()

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder \
    .appName("SparkSQL") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .getOrCreate()

25/05/29 14:07:59 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.Con

In [7]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

In [8]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")

In [10]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""
SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR(date)
ORDER BY year
""").show()

+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [11]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""
SELECT date_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
""").show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [12]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""
SELECT date_built, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
""").show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [13]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
|  51|788128.21|
|  52|733780.26|
|  53| 755214.8|
|  54|798684.82|
|  55|771153.32|
|  56| 718176.4|
|  57| 734340.5|
|  58|759764.65|
|  59| 791453.0|
|  60|754939.65|
|  61|746877.59|
|  62|759150.14|
|  63|711614.55|
|  64|767036.67|
|  65|736679.93|
|  66| 712475.0|
|  67|737970.96|
|  68|716785.44|
|  69|750537.94|
|  70|695865.58|
+----+---------+
only showing top 20 rows

--- 0.18397068977355957 seconds ---


In [14]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [15]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [16]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("""
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view
""").show()

print("--- %s seconds (cached) ---" % (time.time() - start_time))


+----+---------+
|view|avg_price|
+----+---------+
|  51|788128.21|
|  52|733780.26|
|  53| 755214.8|
|  54|798684.82|
|  55|771153.32|
|  56| 718176.4|
|  57| 734340.5|
|  58|759764.65|
|  59| 791453.0|
|  60|754939.65|
|  61|746877.59|
|  62|759150.14|
|  63|711614.55|
|  64|767036.67|
|  65|736679.93|
|  66| 712475.0|
|  67|737970.96|
|  68|716785.44|
|  69|750537.94|
|  70|695865.58|
+----+---------+
only showing top 20 rows

--- 0.1313319206237793 seconds (cached) ---


In [17]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [18]:
# 11. Read the formatted parquet data.
parquet_df = spark.read.parquet("home_sales_partitioned")

In [19]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("parquet_home_sales")

In [20]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM parquet_home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view
""").show()

print("--- %s seconds (parquet) ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
|  51|788128.21|
|  52|733780.26|
|  53| 755214.8|
|  54|798684.82|
|  55|771153.32|
|  56| 718176.4|
|  57| 734340.5|
|  58|759764.65|
|  59| 791453.0|
|  60|754939.65|
|  61|746877.59|
|  62|759150.14|
|  63|711614.55|
|  64|767036.67|
|  65|736679.93|
|  66| 712475.0|
|  67|737970.96|
|  68|716785.44|
|  69|750537.94|
|  70|695865.58|
+----+---------+
only showing top 20 rows

--- 0.30770373344421387 seconds (parquet) ---


In [21]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

DataFrame[]

In [22]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")

False